In [172]:
from requests import RequestException
import requests
import json
from lxml import etree
import time



class Tieba(object):

    def __init__(self, name):
        self.name = name
        self.url_temp = "http://c.tieba.baidu.com/f?ie=utf-8&kw=" +\
                        requests.utils.quote(self.name) +\
                        "&fr=search&pn={}"
        self.headers = {
            "Cookie": "BAIDUID=433DB4AAED0290A1196E65E26551F041:FG=1; BIDUPSID=433DB4AAED0290A1196E65E26551F041; PSTM=1560467964; BDUSS=dnfn5zZnE3eHNlMkMwdjlLR3dUVH56WDR1OHhVQWY1UTBZZjFrSUhhMm9WM3RkRVFBQUFBJCQAAAAAAAAAAAEAAADTpYnPaHR3b28yMjIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAKjKU12oylNdVT; TIEBAUID=8def7439658107cd1106f38b; STOKEN=ffce0822077b028660923867c7aec6d035e7a63adf151b67759a76671085058f; TIEBA_USERTYPE=744b29c1d10868fb821f4b1b; locale=zh; IS_NEW_USER=13f77af84c1af1678ec28563; BAIDU_WISE_UID=wapp_1568616371161_183; H_WISE_SIDS=134725_124610_132921_100805_135964_136285_128065_135948_120155_132909_131247_132378_131518_118882_118865_118855_118830_118798_107312_133352_129655_136193_132250_127025_128967_135307_135813_132551_133287_135432_135874_129643_131423_134600_136019_135592_110085_136144_127969_131754_131951_135672_135458_127417_135870_135036_134383_134353_135834_136259_136077; BDRCVFR[feWj1Vr5u3D]=I67x6TjHwwYf0; delPer=0; PSINO=7; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; H_PS_PSSID=29654_1423_21087_29523_29521_29720_29567_29220_26350_22158; 3481904595_FRSVideoUploadTip=1; USER_JUMP=-1; CLIENTWIDTH=414; CLIENTHEIGHT=736; LASW=414; SEENKW=%E6%9D%8E%E6%AF%85%23%C0%EE%D2%E3; Hm_lvt_98b9d8c2fd6608d564bf2ac2ae642948=1568764717,1568765525,1568765532,1568765535; mo_originid=2; Hm_lpvt_98b9d8c2fd6608d564bf2ac2ae642948=1568765836",
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1"
        }
        
    def parse_url(self, url):
        """解析url，获取html"""
        try:
            print(url)
            r = requests.get(url, headers=self.headers)
            if r.status_code == 200:
                print('获取到网页')
                return r.text
            return None
        except RequestException:
            print('请求失败')
            return None
        except Exception as e:
            print("Error is " + e)
            return None
    
    def get_content_list(self, html_str):
        """提取数据"""
        html = etree.HTML(html_str)
        li_list = html.xpath('//ul[@id="frslistcontent"]/li[@class="tl_shadow tl_shadow_new "]')
        content_list = []
        for li in li_list:
            item = {}
            item['author'] = li.xpath('./div//span[@class="ti_author"]/text()')[0].strip() if len(li.xpath('./div//span[@class="ti_author"]/text()'))>0 else None
            item['title'] = li.xpath('./a//div[@class="ti_title"]/span[last()]/text()') if len(li.xpath('./a//div[@class="ti_title"]/span[last()]'))>0 else None
            item['href'] = "http://c.tieba.baidu.com" + li.xpath('./a[@class="j_common ti_item "]/@href')[0] if len(li.xpath('./a[@class="j_common ti_item "]/@href'))>0 else None
            content_list.append(item)
        return content_list

    def save_data(self, content_list):
        file_path = "../crawl_file/贴吧爬虫-" + self.name  + ".txt"
        with open(file_path, "a+", encoding='utf-8') as f:
            for content in content_list:
                f.write(json.dumps(content, ensure_ascii=False))
                f.write('\n')
        print('保存成功')
        
        
    def run(self):
        pn = 0
        while True:
            # 1.构造url
            url = self.url_temp.format(pn)
            # 2.发送请求，获取html
            html_str = self.parse_url(url)
            # 3.提取数据
            content_list = self.get_content_list(html_str)
            # 4.保存数据
            self.save_data(content_list)
            # 设置退出条件
            if pn > 3000:
                break
            # 5.构造下一个url,进入循环1-4
            pn += 30
            # 设置sleep，防止被反爬
            time.sleep(2)
        

In [174]:
if __name__ == "__main__":
    tieba = Tieba('LOL')
    html_str = tieba.run()

http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=0
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=30
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=60
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=90
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=120
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=150
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=180
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=210
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=240
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=270
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=300
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=330
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=360
获取到网页
保存成功
http://c.tieba.baidu.com/f?ie=utf-8&kw=LOL&fr=search&pn=390
获取到网页
保存成功
http://c.ti

KeyboardInterrupt: 